# Basin BC: ICESat Analysis

## Purpose of this notebook

- Analyze the ICESat tracks from Basin BC, Academy of Science Ice Cap, Severnaya Zemlya. 

## Background

- There are multiple orbital tracking over Basin BC, but Track 154 is the only one worth exploring further:
  - It contains multiple tracks from 2003 to 2009.
  - It is the closest track to the collapsed portion of basin BC.

## What we have done prior to this notebook

- We have made a figure showing elevation profiles of track 154 from different times, with a map inset showing the profile location. (Figure 3 of the term paper)
  - However, these profiles are not properly aligned, so the changes shown on the figure may merely indicate different ice surface height at different locations. Even though each track is close to each other, there is still a maximum distance of a few hunderd meters, enough for meters of different in ice height.

## Goal of this notebook

- Align all the tracks of Track 154 to the same geographical location so that the profile plot accurately shows the elevation change over time.
- We will do this as per figure 2 of Moholdt et al (2010) suggests.

### Workflow:

- Identify tracks
  - There are 12 tracks: 2003-10-30, 2004-03-02, 2004-06-01, 2004-10-18, 2005-03-04, 2005-06-03, 2006-11-08, 2007-03-25, 2007-10-16, 2008-03-02, 2008-11-28, 2009-03-23
  - **2008-03-02** is our **reference track**
  
- Compare the ICESat elevations with ArcticDEM elevations (i.e. sampling ArcticDEM strips at ICESat point locations), and show them on a figure

### Possible Github names

- ParallelPointFinder?
- TrackAlign
- Track-A-Line
- icesat align tracks
- align tracks
- SnowShoe

### (1) Import necessary Python modules

In [1]:
# make all plots interactive
%matplotlib widget

### ==== Load necessary packages ===

# for processing rasters
import rasterio
from rasterio.plot import show
# for processing ICESat data
import pandas as pd
import geopandas as gpd
# for plotting
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
# for vector geospatial processes
from shapely.geometry import Point
import numpy as np
from scipy.interpolate import interp1d
from scipy.stats import linregress
from datetime import datetime

# import custom functions we write from a separated Python file
from utility import points_in_polygon, find_tri_vertices, find_perp_coord

### (2) Specify input data

You can change the path to other files if you find them more suitable.

In [2]:
# ICESat collection
icesat_file = '/data/whyj/Projects/Cryosphere/Data/MOHOLDT_ICESAT/SevernayaZ_GLA06_all19_AoSIceCapOnly_EPSG3413.shp'  # EPSG 3413

# Reference DEM (now using ArcticDEM but there might be better files online we can use - take a look)
reference_dem_file = '/data/whyj/Projects/Leena_AoS_SVZ/Data/ArcticDEM/Rel7_Mosaic_32m/SVZ_ArcticDEM_32m.tif'         # EPSG 3413

# Extent of Basin BC, from RGI 6.0
basinBC_extent_file = '/13t1/mpguest/lps57/RGI/09_rgi60_RussianArctic_EPSG3413_singlepolygonC.shp'                    # EPSG 3413

# Basemap from the ArcticDEM dhdt
raster_basemap_file = '/13t1/mpguest/lps57/arcticDEM3.0/dhdt_ASI/outputmerges/allout.tif'                             # EPSG 3413

### (3) Load data into appropriate Python objects

In [3]:
# ICESat (point data) --> Geopandas GeoDataFrame 
icesat = gpd.read_file(icesat_file)

# Reference DEM (raster data) --> RasterIO Dataset
reference_dem = rasterio.open(reference_dem_file)

# Basin BC extent (polygon data) --> Geopandas GeoDataFrame 
basinBC_extent = gpd.read_file(basinBC_extent_file)

# dh/dt Basemap (raster data) --> RasterIO Dataset
raster_basemap = rasterio.open(raster_basemap_file)

You can check the content of a `GeoDataFrame`:

In [4]:
icesat

,Time,Longitude,Latitude,East,North,H_ell,Geoid,Gain,SN,Energy,Reflect,Recordnr,Shotnr,Stripnr,Reftracknr,H_orto,Region,geometry
0,20081124,95.044982,80.353798,426070.473860,8.923592e+06,584.289702,3.006702,69,3.071438,1.36,1.020408,931398225,14,680,100,581.283000,0,POINT (672575.941 802823.967)
1,20081124,95.041097,80.355201,426008.622594,8.923753e+06,585.725702,3.009010,73,3.135797,1.40,1.020408,931398225,15,680,100,582.716692,0,POINT (672532.100 802661.068)
2,20081124,95.037206,80.356605,425946.688240,8.923915e+06,587.311702,3.011317,72,3.120285,1.40,1.020408,931398225,16,680,100,584.300385,0,POINT (672488.254 802498.026)
3,20081124,95.033305,80.358011,425884.603861,8.924076e+06,588.700702,3.013625,67,3.078991,1.38,1.020408,931398225,17,680,100,585.687077,0,POINT (672444.388 802334.709)
4,20081124,95.029392,80.359419,425822.332277,8.924238e+06,590.200702,3.015932,64,3.001763,1.35,1.020408,931398225,18,680,100,587.184769,0,POINT (672400.532 802171.093)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
134015,20081124,95.064357,80.346786,426379.137836,8.922787e+06,576.455703,2.995164,85,3.151829,1.38,1.020408,931398225,9,680,100,573.460538,0,POINT (672795.358 803637.769)
134016,20081124,95.060490,80.348192,426317.539274,8.922948e+06,578.064703,2.997472,76,3.146286,1.45,1.020408,931398225,10,680,100,575.067231,0,POINT (672751.148 803474.782)
134017,20081124,95.056619,80.349594,426255.856952,8.923109e+06,579.624702,2.999779,64,3.056003,1.31,1.020408,931398225,11,680,100,576.624923,0,POINT (672707.255 803312.092)
134018,20081124,95.052744,80.350995,426194.113541,8.923270e+06,581.242702,3.002087,57,3.022316,1.33,1.020408,931398225,12,680,100,578.240615,0,POINT (672663.470 803149.449)


And here's how to visualize a `Rasterio` object. Note that you can use the control panel in the left to zoom in, zoom out, and save figure.

In [5]:
fig, ax = plt.subplots(1,figsize=(5,5)) 

# Plot colorbar
image_data = reference_dem.read(1)
image_hidden = ax.imshow(image_data, cmap='viridis', clim=[0, 800])
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="5%", pad=0.05)
fig.colorbar(image_hidden, cax=cax, label='Elevation (m)')

show(reference_dem, ax=ax, clim=[0, 800])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<AxesSubplot:>

### (4) Visualize the input data

In [6]:
fig, ax = plt.subplots(1,figsize=(10,10))    # creates an empty figure with a desired size -- 10x10 inches, I *think*
# where  fig is the figure handle
# and     ax is the axis (plotting area) handle

# plot dh/dt basemap
show(raster_basemap, ax=ax, cmap='RdBu', clim=[-5, 5])
# plot polygon boundary
basinBC_extent.boundary.plot(ax=ax, color='k')

# plot ICESat points 
icesat.plot(ax=ax, markersize=3)
# plot ICESat track 154 in a different color (highlight)
icesat[icesat["Reftracknr"] == 154].plot(ax=ax, markersize=3, color='xkcd:purple')

# Plot colorbar
image_data = raster_basemap.read(1)
image_hidden = ax.imshow(image_data, cmap='RdBu', clim=[-5, 5])
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="5%", pad=0.05)
fig.colorbar(image_hidden, cax=cax, label='Elevation change (m/yr)')

# Set axis x boundary
ax.set_xlim([655000, 680000])
# Set axis y boundary
ax.set_ylim([795000, 830000])
# Set axis title
ax.set_title('Basin BC of AoS with ICESat tracks | Basemap: ArcticDEM dh/dt')
ax.set_xlabel('Eastings(m)')
ax.set_ylabel('Northings (m)')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0, 0.5, 'Northings (m)')

### (5) Locate points on basin BC

Using the `points_in_polygon` function, we can get the indices indicating the points on Basin BC. 

We add this information as an additional attribute of the icesat `GeoDataFrame` object:

In [7]:
idx = points_in_polygon(icesat, basinBC_extent_file)
icesat['on_BC'] = idx

# Now there's one additional field "on_BC" in the right:
icesat

,Time,Longitude,Latitude,East,North,H_ell,Geoid,Gain,SN,Energy,Reflect,Recordnr,Shotnr,Stripnr,Reftracknr,H_orto,Region,geometry,on_BC
0,20081124,95.044982,80.353798,426070.473860,8.923592e+06,584.289702,3.006702,69,3.071438,1.36,1.020408,931398225,14,680,100,581.283000,0,POINT (672575.941 802823.967),False
1,20081124,95.041097,80.355201,426008.622594,8.923753e+06,585.725702,3.009010,73,3.135797,1.40,1.020408,931398225,15,680,100,582.716692,0,POINT (672532.100 802661.068),False
2,20081124,95.037206,80.356605,425946.688240,8.923915e+06,587.311702,3.011317,72,3.120285,1.40,1.020408,931398225,16,680,100,584.300385,0,POINT (672488.254 802498.026),False
3,20081124,95.033305,80.358011,425884.603861,8.924076e+06,588.700702,3.013625,67,3.078991,1.38,1.020408,931398225,17,680,100,585.687077,0,POINT (672444.388 802334.709),False
4,20081124,95.029392,80.359419,425822.332277,8.924238e+06,590.200702,3.015932,64,3.001763,1.35,1.020408,931398225,18,680,100,587.184769,0,POINT (672400.532 802171.093),False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
134015,20081124,95.064357,80.346786,426379.137836,8.922787e+06,576.455703,2.995164,85,3.151829,1.38,1.020408,931398225,9,680,100,573.460538,0,POINT (672795.358 803637.769),False
134016,20081124,95.060490,80.348192,426317.539274,8.922948e+06,578.064703,2.997472,76,3.146286,1.45,1.020408,931398225,10,680,100,575.067231,0,POINT (672751.148 803474.782),False
134017,20081124,95.056619,80.349594,426255.856952,8.923109e+06,579.624702,2.999779,64,3.056003,1.31,1.020408,931398225,11,680,100,576.624923,0,POINT (672707.255 803312.092),False
134018,20081124,95.052744,80.350995,426194.113541,8.923270e+06,581.242702,3.002087,57,3.022316,1.33,1.020408,931398225,12,680,100,578.240615,0,POINT (672663.470 803149.449),False


In [8]:
# Subsetting the icesat object
icesat_BC = icesat[icesat['on_BC'] == True]
# Showing only Time, H_ell (ellipsoidal height), and location attributes. AVOID USING the numbers from the East and North fields because they are NOT in EPSG:3413.
icesat_BC[['Time', 'H_ell', 'geometry']]

,Time,H_ell,geometry
2861,20030220,2.310713,POINT (668473.298 825832.574)
2863,20030220,12.448713,POINT (668432.390 825667.932)
2865,20030220,35.531713,POINT (668391.475 825503.431)
2867,20030220,63.016713,POINT (668350.606 825339.168)
2869,20030220,91.216713,POINT (668309.643 825175.115)
...,...,...,...
134008,20081124,562.875703,POINT (673101.240 804774.836)
134009,20081124,564.977703,POINT (673058.085 804612.713)
134010,20081124,566.919703,POINT (673014.897 804450.754)
134011,20081124,568.992703,POINT (672971.578 804288.757)


**FYI**: ellipsoidal height (`H_ell`) vs. orthometric height (`H_orto`) vs. geoid height (`Geoid`):

<img src="https://www.esri.com/news/arcuser/0703/graphics/geoid2_lg.gif" />

### (6) Make a field for the date object (optional)

Conver timestemp (in float format) to datetime obj. This is optional because there is already a `Time` field telling you the date, but if you want the time string better formated in legends, outputs, etc., you will probably need this field.

In [9]:
icesat_BC['Time_obj'] = pd.to_datetime(icesat_BC['Time'].astype(str))
icesat_BC[['Time_obj', 'H_ell', 'geometry']]

/data/mpguest/Software/anaconda3/envs/carst/lib/python3.6/site-packages/geopandas/geodataframe.py:853: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)


,Time_obj,H_ell,geometry
2861,2003-02-20,2.310713,POINT (668473.298 825832.574)
2863,2003-02-20,12.448713,POINT (668432.390 825667.932)
2865,2003-02-20,35.531713,POINT (668391.475 825503.431)
2867,2003-02-20,63.016713,POINT (668350.606 825339.168)
2869,2003-02-20,91.216713,POINT (668309.643 825175.115)
...,...,...,...
134008,2008-11-24,562.875703,POINT (673101.240 804774.836)
134009,2008-11-24,564.977703,POINT (673058.085 804612.713)
134010,2008-11-24,566.919703,POINT (673014.897 804450.754)
134011,2008-11-24,568.992703,POINT (672971.578 804288.757)


### (7) Another plot showing the input data (optional)

This one shows the ICESAE data that are over Basic BC.

In [10]:
fig,ax = plt.subplots(1,figsize=(10,10))
show(raster_basemap, ax=ax, cmap='RdBu', clim=[-5, 5])
basinBC_extent.boundary.plot(ax=ax, color='k')
icesat[icesat['on_BC'] == True].plot(ax=ax, markersize=3)

# Plot colorbar
image_data = raster_basemap.read(1)
image_hidden = ax.imshow(image_data, cmap='RdBu', clim=[-5, 5])
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="5%", pad=0.05)
fig.colorbar(image_hidden, cax=cax, label='Elevation change (m/yr)')

# set x limit, y limit, and title
ax.set_xlim([655000, 680000])
ax.set_ylim([795000, 830000])
ax.set_title('Basinc BC of AoS | Basemap: ArcticDEM dh/dt')
ax.set_ylabel('Eastings (m)')
ax.set_xlabel('Northings (m)')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 0, 'Northings (m)')

### (8) Subset ICESat data

We only select the data from Track 154:

In [11]:
icesat_154_BC = icesat_BC.loc[icesat_BC['Reftracknr'] == 154]
icesat_154_BC

,Time,Longitude,Latitude,East,North,H_ell,Geoid,Gain,SN,Energy,Reflect,Recordnr,Shotnr,Stripnr,Reftracknr,H_orto,Region,geometry,on_BC,Time_obj
8330,20081128,95.637402,80.364087,437197.689308,8.924041e+06,532.892701,3.035676,102,2.699504,0.59,1.020408,932965400,21,-68,154,529.857026,0,POINT (663527.474 808867.532),True,2008-11-28
8332,20081128,95.633488,80.362671,437115.516893,8.923888e+06,531.618701,3.033625,126,2.597969,0.51,1.020408,932965400,22,-68,154,528.585077,0,POINT (663680.685 808941.599),True,2008-11-28
8334,20081128,95.629569,80.361257,437033.243167,8.923734e+06,530.128702,3.031573,203,2.750826,0.47,1.020408,932965400,23,-68,154,527.097128,0,POINT (663833.841 809015.423),True,2008-11-28
8336,20081128,95.625644,80.359846,436950.855950,8.923581e+06,528.186702,3.029522,250,2.770852,0.44,1.020408,932965400,24,-68,154,525.157179,0,POINT (663986.887 809088.908),True,2008-11-28
8338,20081128,95.621716,80.358436,436868.398292,8.923429e+06,526.770702,3.027471,250,2.889388,0.58,1.020408,932965400,25,-68,154,523.743231,0,POINT (664139.921 809162.256),True,2008-11-28
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
123649,20031030,95.335976,80.252163,430787.939634,8.911913e+06,350.522710,2.862864,13,3.352343,24.73,2100.000000,131373536,2,-98,154,347.659846,0,POINT (675565.846 814763.395),True,2003-10-30
123650,20031030,95.332150,80.250747,430705.794987,8.911760e+06,349.008710,2.859018,13,3.388071,24.32,2100.000000,131373536,3,-98,154,346.149692,0,POINT (675718.850 814837.179),True,2003-10-30
123651,20031030,95.328313,80.249333,430623.436421,8.911607e+06,347.726710,2.855172,13,3.329083,24.51,2100.000000,131373536,4,-98,154,344.871538,0,POINT (675871.884 814910.650),True,2003-10-30
125889,20060308,95.639052,80.363116,437222.207637,8.923931e+06,532.196701,3.033112,250,2.314344,0.22,1.258855,502832314,32,-42,154,529.163590,0,POINT (663571.344 808968.520),True,2006-03-08


And here's another figure that shows the data from Track 154. We plot the reference track (2008-03-02) using a different color. All the other tracks will be aligned to that reference positions.

In [12]:
fig,ax = plt.subplots(1,figsize=(10,6))
# according to https://stackoverflow.com/questions/61327088/rio-plot-show-with-colorbar
# use imshow so that we have something to map the colorbar to
image_data = raster_basemap.read(1)
image_hidden = ax.imshow(image_data, cmap='RdBu', clim=[-5, 5])

# plot on the same axis with rio.plot.show
image = show(raster_basemap, ax=ax, cmap='RdBu', clim=[-5, 5])
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="5%", pad=0.05)
fig.colorbar(image_hidden, cax=cax, label='Elevation change (m/yr)')

# according to https://stackoverflow.com/questions/18195758/set-matplotlib-colorbar-size-to-match-graph
# create an axes on the right side of ax. The width of cax will be 5%
# of ax and the padding between cax and ax will be fixed at 0.05 inch.

basinBC_extent.boundary.plot(ax=ax, color='k')
# We need multiple plotting command for different colors 
icesat_154_BC.loc[icesat_154_BC['Time'] == 20080302].plot(ax=ax, markersize=6, label='Reference track', color='xkcd:blue')
icesat_154_BC.loc[icesat_154_BC['Time'] != 20080302].plot(ax=ax, markersize=3, label='Other tracks', color='xkcd:green')
#
ax.set_xlim([663000, 676000])
ax.set_ylim([808000, 816000])
ax.set_title('Basinc BC of AoS | Basemap: ArcticDEM dh/dt')
ax.set_xlabel('Eastings (m)')
ax.set_ylabel('Northings (m)')
ax.legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### (9) Find the location along the reference track for each point to be projected on

For each point that is not from Track 2008-03-02, We want to select the other two points from Track 2008-03-02 so they form a triangle. And we find the coordinates on the reference track for the target point to be projected based on the coordinates of the triangle vertices.

We need a lookup table to find out two closest points from the reference track for each point not from Track 2008-03-02. Because these points are ordered by their `Recordnr` and `Shotnr`, we can achieve this by writing down the corresponding `Recordnr` and `Shotnr`  numbers for each track. I (Whyjay) did this on QGIS.

In [13]:
tripoints_lookup = {20080302: (816048397, 1),
                    20050603: (382895576, 40), 
                    20070325: (668068898, 14), 
                    20081128: (932965400, 32), 
                    20071016: (756511563, 16), 
                    20041018: (284098826, 4), 
                    20061108: (608532031, 38), 
                    20090323: (982579441, 31), 
                    20031030: (131373526, 14), 
                    20040302: (184870482, 40), 
                    20040601: (224130404, 10), 
                    20050304: (343635694, 8)}
tripoints_lookup[20080302]

(816048397, 1)

#### (9-1) Test run

We subset a track (2003-10-30) and see how this works.

In [14]:
# Subsetting the data
test_s = icesat_154_BC.loc[icesat_154_BC['Time'] == 20031030]

# Plot data
fig,ax = plt.subplots(1,figsize=(10,10))
show(raster_basemap, ax=ax, cmap='RdBu', clim=[-5, 5])
basinBC_extent.boundary.plot(ax=ax, color='k')
icesat_154_BC.loc[icesat_154_BC['Time'] == 20080302].plot(ax=ax, markersize=6, label='Reference track', color='xkcd:blue')
icesat_154_BC.loc[icesat_154_BC['Time'] != 20080302].plot(ax=ax, markersize=3, label='Other tracks', color='xkcd:green')
test_s.plot(ax=ax, markersize=10, label='Test track', color='xkcd:brown')

# Plot colorbar
image_data = raster_basemap.read(1)
image_hidden = ax.imshow(image_data, cmap='RdBu', clim=[-5, 5])
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="5%", pad=0.05)
fig.colorbar(image_hidden, cax=cax, label='Elevation change (m/yr)')


# identifier for where to show the triangle points
purple_pt_idx = 30

# loop records through test_s
for idx, record in test_s.iterrows():
    if record['Time'] != 20080302:
        tri_pt1, tri_pt2, tri_pt3 = find_tri_vertices(record, tripoints_lookup, icesat_154_BC)
        # print(tri_pt1, tri_pt2, tri_pt3)
        if record['Shotnr'] == purple_pt_idx:
            # plot the located triangle points in purple
            tmp = gpd.GeoSeries([tri_pt1, tri_pt2, tri_pt3])
            tmp.plot(ax=ax, markersize=15, label='Selected triangle', color='xkcd:purple')
            
ax.set_xlim([663000, 676000])
ax.set_ylim([808000, 816000])
ax.set_title('Basin BC of AoS | Basemap: ArcticDEM dh/dt')
ax.set_xlabel('Eastings (m)')
ax.set_ylabel('Northings (m)')
ax.legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

#### (9-2) Now let's do the same thing for all the Track 154 data

To store the new data, we need a different `GeoDataFrame`.

In [15]:
# This is the field name list for the new GeoDataFrame.
fields = ['Time', 'East', 'North', 'H_ell', 'Recordnr', 'Shotnr', 'Reftracknr', 'geometry', 'Time_obj']

# This is where we store the output data from the loop. It's empty now but new results from each loop will be appended here
icesat_BC_processed_list = []

# loop records through icesat_154_BC
for idx, record in icesat_154_BC.iterrows():
    # if not reference frame, do the process
    if record['Time'] != 20080302:
        # (1) find triangle points
        tri_pt1, tri_pt2, tri_pt3 = find_tri_vertices(record, tripoints_lookup, icesat_154_BC)
        # (2) find the perpendicular base coordinates
        perp_pt = find_perp_coord(tri_pt1, tri_pt2, tri_pt3)
        # (3) if all points are vaild (i.e., not NaN), sample the DEM height for the original and the perpendicular base points
        if sum(np.isnan(perp_pt)) == 0:
            dem_sample_pts = record['geometry'].coords[:]
            dem_sample_pts.append(tuple(perp_pt))
            dem_sample_h = []
            for val in reference_dem.sample(dem_sample_pts):
                # print(float(val))
                dem_sample_h.append(float(val))
            # (4) calculate elevation difference between these two points
            h_difference = dem_sample_h[1] - dem_sample_h[0]
            # (5) add the elevation difference to the ICESat height
            perp_pt_h = record['H_ell'] + h_difference
            # (6) save data to icesat_BC_processed_list
            row = [record['Time'], perp_pt[0], perp_pt[1], perp_pt_h, record['Recordnr'], record['Shotnr'], record['Reftracknr'], Point(perp_pt), record['Time_obj']]
            icesat_BC_processed_list.append(row)
    # if reference frame, copy the original data
    else:
        row = [record['Time'], record['geometry'].x, record['geometry'].y, record['H_ell'], record['Recordnr'], record['Shotnr'], record['Reftracknr'], record['geometry'], record['Time_obj']]
        icesat_BC_processed_list.append(row)

# Now convert icesat_BC_processed_list to GeoDataFrame!
icesat_BC_processed_gpd = gpd.GeoDataFrame(icesat_BC_processed_list, columns=fields)
icesat_BC_processed_gpd


,Time,East,North,H_ell,Recordnr,Shotnr,Reftracknr,geometry,Time_obj
0,20081128,663776.575459,808737.348848,533.130542,932965400,22,154,POINT (663776.575 808737.349),2008-11-28
1,20081128,663930.555973,808809.716909,531.748941,932965400,23,154,POINT (663930.556 808809.717),2008-11-28
2,20081128,664084.189249,808881.947912,530.333858,932965400,24,154,POINT (664084.189 808881.948),2008-11-28
3,20081128,664238.226942,808954.803659,528.898693,932965400,25,154,POINT (664238.227 808954.804),2008-11-28
4,20081128,664392.007893,809028.226101,526.871507,932965400,26,154,POINT (664392.008 809028.226),2008-11-28
...,...,...,...,...,...,...,...,...,...
929,20031030,675257.321940,814223.943763,364.447478,131373531,39,154,POINT (675257.322 814223.944),2003-10-30
930,20031030,675411.302542,814296.940839,362.546720,131373531,40,154,POINT (675411.303 814296.941),2003-10-30
931,20031030,675565.326253,814370.420822,360.370318,131373536,1,154,POINT (675565.326 814370.421),2003-10-30
932,20031030,675719.089921,814444.173867,358.232274,131373536,2,154,POINT (675719.090 814444.174),2003-10-30


Now all the tracks are aligned to the same line! 

In [16]:
fig,ax = plt.subplots(1,figsize=(10,10))
show(raster_basemap, ax=ax, cmap='RdBu', clim=[-5, 5])
basinBC_extent.boundary.plot(ax=ax, color='k')
icesat_BC_processed_gpd.loc[icesat_BC_processed_gpd['Time'] == 20080302].plot(ax=ax, markersize=6, label='Reference track', color='xkcd:blue')
icesat_BC_processed_gpd.loc[icesat_BC_processed_gpd['Time'] != 20080302].plot(ax=ax, markersize=3, label='Other tracks', color='xkcd:green')

# Plot colorbar
image_data = raster_basemap.read(1)
image_hidden = ax.imshow(image_data, cmap='RdBu', clim=[-5, 5])
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="5%", pad=0.05)
fig.colorbar(image_hidden, cax=cax, label='Elevation change (m/yr)')

ax.set_xlim([663000, 676000])
ax.set_ylim([808000, 816000])
ax.set_xlabel('Eastings (m)')
ax.set_ylabel('Northings (m)')
ax.set_title('Basinc BC of AoS | Basemap: ArcticDEM dh/dt')
ax.legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### (10) Interpolate these points to the reference point locations

Now all the points are along the same line, it is time to interpolate them to where the reference points from Track 2008-03-02 are located.

In [17]:
# First we create a list for available dates 
dategroup_list = icesat_BC_processed_gpd['Time'].unique()
dategroup_list

array([20081128, 20050603, 20071016, 20041018, 20040302, 20061108,
       20080302, 20090323, 20050304, 20070325, 20040601, 20031030])

In [18]:
# Now we create a list of point locations along the reference track.
ref_dategroup = icesat_BC_processed_gpd.loc[icesat_BC_processed_gpd['Time'] == 20080302]
ref_dategroup = ref_dategroup.drop(columns=['Recordnr', 'Shotnr', 'Reftracknr', 'Time_obj'])

# Calculate distance (in meters) from the first point, for plotting purpose. 
ref_x = ref_dategroup['East'].to_numpy()
ref_y = ref_dategroup['North'].to_numpy()
seg_dist = np.sqrt(np.diff(ref_x) ** 2 + np.diff(ref_y) ** 2)
dist = np.zeros(len(seg_dist) + 1)
dist[1:] = np.cumsum(seg_dist)
ref_dategroup['Distance'] = dist
ref_dategroup

,Time,East,North,H_ell,geometry,Distance
453,20080302,663664.697040,808684.824520,534.343701,POINT (663664.697 808684.825),0.000000
454,20080302,663817.034504,808756.343434,533.074701,POINT (663817.035 808756.343),168.290398
455,20080302,663969.286414,808827.926489,531.646702,POINT (663969.286 808827.926),336.530637
456,20080302,664121.425887,808899.454670,530.117702,POINT (664121.426 808899.455),504.645776
457,20080302,664273.558932,808971.546572,528.726702,POINT (664273.559 808971.547),672.995726
...,...,...,...,...,...,...
529,20080302,675284.561201,814236.804092,365.543710,POINT (675284.561 814236.804),12878.977061
530,20080302,675437.596834,814309.417062,363.295710,POINT (675437.597 814309.417),13048.365810
531,20080302,675590.696718,814382.537794,360.994710,POINT (675590.697 814382.538),13218.030821
532,20080302,675743.721555,814455.998459,358.601710,POINT (675743.722 814455.998),13387.774952


In [19]:
# Now loop through each date and interpolate the points based on the reference x coordinates.
icesat_BC_final_gpd = ref_dategroup.copy()

for dategroup_label in dategroup_list:
    if dategroup_label != 20080302:
        dategroup = icesat_BC_processed_gpd.loc[icesat_BC_processed_gpd['Time'] == dategroup_label]
        x = dategroup['East'].to_numpy()
        z = dategroup['H_ell'].to_numpy()
        f = interp1d(x, z, kind='linear', bounds_error=False, fill_value=np.nan)
        z_at_ref = f(ref_x)
        
        interpolated_dategroup = ref_dategroup.copy()
        interpolated_dategroup['Time'] = dategroup_label
        interpolated_dategroup['H_ell'] = z_at_ref
        icesat_BC_final_gpd = icesat_BC_final_gpd.append(interpolated_dategroup)

# add Time_obj column
icesat_BC_final_gpd['Time_obj'] = pd.to_datetime(icesat_BC_final_gpd['Time'].astype(str))
# sort the date table by Time_obj
icesat_BC_final_gpd = icesat_BC_final_gpd.sort_values(by=['Time_obj'])
# convert GeoDataFrame to DataFrame for plotting purpose
icesat_BC_final_pd = pd.DataFrame(icesat_BC_final_gpd)
icesat_BC_final_pd['Distance (km)'] = icesat_BC_final_pd['Distance'] / 1000
icesat_BC_final_pd


,Time,East,North,H_ell,geometry,Distance,Time_obj,Distance (km)
533,20031030,675896.614451,814529.846344,NaN,POINT (675896.614 814529.846),13557.568203,2003-10-30,13.557568
474,20031030,666859.267989,810245.384828,496.856648,POINT (666859.268 810245.385),3555.488669,2003-10-30,3.555489
475,20031030,667012.432080,810319.207535,494.391540,POINT (667012.432 810319.208),3725.515228,2003-10-30,3.725515
476,20031030,667166.182409,810393.248123,492.144306,POINT (667166.182 810393.248),3896.164495,2003-10-30,3.896164
477,20031030,667320.035214,810466.774547,489.928066,POINT (667320.035 810466.775),4066.683763,2003-10-30,4.066684
...,...,...,...,...,...,...,...,...
509,20090323,672214.225708,812798.629449,406.630522,POINT (672214.226 812798.629),9488.418200,2009-03-23,9.488418
510,20090323,672366.172547,812872.545334,403.415896,POINT (672366.173 812872.545),9657.389796,2009-03-23,9.657390
511,20090323,672518.289838,812946.173024,398.914517,POINT (672518.290 812946.173),9826.388928,2009-03-23,9.826389
493,20090323,669782.399944,811606.150690,448.642416,POINT (669782.400 811606.151),6779.916365,2009-03-23,6.779916


### (11) Plotting aligned ICESat Track 154 data

In [20]:
dategroup_list_new = icesat_BC_final_pd['Time'].unique()
# assign colors
colors = plt.cm.viridis(np.linspace(0,0.7,len(dategroup_list_new)))

fig,ax = plt.subplots(1,figsize=(10,5))
# i here controls the color representation
i = 0
for tmp in dategroup_list_new:
    legend_label = '-'.join([str(tmp)[:4], str(tmp)[4:6], str(tmp)[6:]])  # create labels (yyyy-mm-dd) for the legend
    date_group = icesat_BC_final_pd.loc[icesat_BC_final_pd['Time'] == tmp]
    date_group = date_group.sort_values(by=['Distance'])
    date_group.plot(x='Distance (km)', y='H_ell', ax=ax, color=tuple(colors[i]), label=legend_label)
    i += 1
    
plt.xlabel('Distance (km)')
plt.ylabel('Surface elevation (m)')
plt.title('Aligned Track 154 Profiles from 2003-2009')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 1.0, 'Aligned Track 154 Profiles from 2003-2009')

### (12) Calculate dh/dt

Final step: calculate dh/dt for each point on the reference track.

In [21]:
dhdt = ref_dategroup.copy()
dhdt = dhdt.drop(columns=['Time', 'H_ell'])

slopes = []

for idx, record in ref_dategroup.iterrows():
    pt_dist = record['Distance']
    pt_collection = icesat_BC_final_gpd.loc[icesat_BC_final_gpd['Distance'] == pt_dist]
    date_collection = pt_collection['Time_obj'].map(datetime.toordinal).to_numpy()
    elev_collection = pt_collection['H_ell'].to_numpy()
    slope, intercept, r, p, se = linregress(date_collection, elev_collection)
    # print(slope, intercept, r, p, se)
    slope *= 365.25
    slopes.append(slope)
    
dhdt['dhdt'] = slopes
dhdt

,East,North,geometry,Distance,dhdt
453,663664.697040,808684.824520,POINT (663664.697 808684.825),0.000000,NaN
454,663817.034504,808756.343434,POINT (663817.035 808756.343),168.290398,NaN
455,663969.286414,808827.926489,POINT (663969.286 808827.926),336.530637,-0.279089
456,664121.425887,808899.454670,POINT (664121.426 808899.455),504.645776,-0.228962
457,664273.558932,808971.546572,POINT (664273.559 808971.547),672.995726,-0.243740
...,...,...,...,...,...
529,675284.561201,814236.804092,POINT (675284.561 814236.804),12878.977061,0.189808
530,675437.596834,814309.417062,POINT (675437.597 814309.417),13048.365810,0.249121
531,675590.696718,814382.537794,POINT (675590.697 814382.538),13218.030821,0.181252
532,675743.721555,814455.998459,POINT (675743.722 814455.998),13387.774952,0.102046


### (13) Plot dh/dt on the Basin BC map

In [22]:
fig,ax = plt.subplots(1,figsize=(11,6))
show(raster_basemap, ax=ax, cmap='RdBu', clim=[-5, 5])
basinBC_extent.boundary.plot(ax=ax, color='k')
dhdt.plot(ax=ax, markersize=15, column='dhdt', label='dh/dt (m/yr)', legend=True)
ax.set_xlim([663000, 676000])
ax.set_ylim([808000, 816000])
# plt.title('Basinc BC of AoS | Basemap: ArcticDEM dh/dt | Points: ICESat dh/dt')
ax.set_title('Basin BC of AoS | Basemap: ArcticDEM dh/dt | Points: ICESat dh/dt')
ax.set_xlabel('Eastings (m)')
ax.set_ylabel('Northings (m)')
ax.legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [23]:
fig,axs = plt.subplots(2,3,figsize=(11,6))
axs[0, 0].plot(3,4,'.')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …